In [4]:
import sqlite3
import json

def get_dashboard_data(channel_name, start_time):

    channel_name = '\'' + channel_name + '\''
    start_time = '\'' + start_time + '\''


    conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')

    # Calculate Average Views and Total Messages
    avg_views_total_messages_query = '''
                        select count(*) as total_messages, 
                        round(avg(viewer_count)) as avg_viewers,
                        max(stream_length) as stream_length_minutes,
                        max(viewer_count) as max_viewers
                        from chats_table_sentiment
                        where channel_name = {} and stream_date = {}
                        '''.format(channel_name, start_time)
    
    sentiment_metrics_query =  '''
                            select
                            round(avg(message_sentiment),2) as average_sentiment,
                            round(max(message_sentiment),2) as max_sentiment,
                            round(min(message_sentiment),2) as min_sentiment
                            from chats_table_sentiment
                            where channel_name = {} 
                            and stream_date = {}
                            and message_sentiment is not NULL
                            '''.format(channel_name, start_time)

                            
    
    # Calculate followers at start and end of stream
    followers_subscribers_query =  '''
                        with starting_followers as (
                            select follower_count, subscriber_count
                            from chats_table_sentiment
                            where channel_name = {} and stream_date = {} 
                            order by date asc
                            limit 1)

                        ,ending_followers as (
                            select follower_count, subscriber_count
                            from chats_table_sentiment
                            where channel_name = {} and stream_date = {}
                            order by date desc
                            limit 1
                            )
                        
                        select * from starting_followers
                        UNION ALL select * from ending_followers
                        
                        '''.format(channel_name, start_time, channel_name, start_time)
    
    
    cursor_obj = conn.cursor()
    cursor_obj.execute(avg_views_total_messages_query)
    averages_output = cursor_obj.fetchall()
    
    cursor_obj = conn.cursor()
    cursor_obj.execute(sentiment_metrics_query)
    sentiment_output = cursor_obj.fetchall()
    
    cursor_obj.execute(followers_subscribers_query)
    followers_subscribers = cursor_obj.fetchall()
    followers_gained = followers_subscribers[1][0] - followers_subscribers[0][0]
    followers_pct_change = round(abs(followers_gained) / followers_subscribers[0][0],4)*100
    subscribers_gained = followers_subscribers[1][1] - followers_subscribers[0][1]
    subscriber_pct_change = round(abs(subscribers_gained) / followers_subscribers[0][1],4) *100
    
    recommender_query = '''with grouped_stream as (
                    select channel_name, 
                    stream_topic, 
                    avg(viewer_count) as avg_viewers, 
                    max(follower_count) as max_followers,
                    min(follower_count) as min_followers,
                    max(subscriber_count) as max_subscribers,
                    min(subscriber_count) as min_subscribers,
                    max(stream_length) as max_minutes,
                    min(stream_length) as min_minutes
                    from chats_table_sentiment
                    where channel_name = {}
                    group by channel_name, stream_topic
                    )
            select channel_name,
            stream_topic,
            avg_viewers, 
            (max_followers - min_followers) as followers_gained,
            (max_subscribers - min_subscribers) as subscribers_gained,
            (max_minutes - min_minutes) as time
            from grouped_stream
                    '''.format(channel_name)
    
    
    
    conn.commit()
    conn.close()  


    return json.dumps([{
        'total_messages': averages_output[0][0],
        'average_viewers': averages_output[0][1],
        'new_followers': followers_gained,
        'max_viewers': averages_output[0][3],
        'average_sentiment': sentiment_output[0][0],
        'max_sentiment': sentiment_output[0][1],
        'min_sentiment': sentiment_output[0][2],
        'follower_pct_change': followers_pct_change,
        'subscribers_gained': subscribers_gained,
        'subscriber_pct_change': subscriber_pct_change
    }])

In [ ]:


    

    '''with grouped_stream as (
                    select channel_name, 
                    stream_topic, 
                    avg(viewer_count) as avg_viewers, 
                    max(follower_count) as max_followers,
                    min(follower_count) as min_followers,
                    max(subscriber_count) as max_subscribers,
                    min(subscriber_count) as min_subscribers,
                    max(stream_length) as max_minutes,
                    min(stream_length) as min_minutes
                    from chats_table
                    where channel_name = {}
                    group by channel_name, stream_topic
                    )
            select channel_name,
            stream_topic,
            avg_viewers, 
            (max_followers - min_followers) as followers_gained,
            (max_subscribers - min_subscribers) as subscribers_gained,
            (max_minutes - min_minutes) as time
            from grouped_stream
            '''.format(channel_name)

In [62]:
import sqlite3
import pandas as pd
import numpy as np

def recommendation(row):
    '''
    Basic recommendation engine. Each normalized category divided by the seconds spent times the weight. 
    
    Current weight configuration: 
    35% Sentiment Value
    30% Subs Gained
    25% Followers Gained
    10% Views
    '''
    value = ((0.10 * (row['avg_viewers'])/row['time']) 
            + (0.25 * (row['followers_change'])/row['time']) 
            + (0.30 * (row['subscriber_change'])/row['time']) 
            + (0.35 * (row['average_sentiment'])/row['time']))
    return value


def recommender_engine(channel_name, start_time):

    channel_name = '\'' + channel_name + '\''
    start_time = '\'' + start_time + '\''
    conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')


    distinct_topics_query = '''
                        select distinct stream_topic 
                        from chats_table_sentiment
                        where channel_name = {} 
                        and stream_date = {}
                    '''.format(channel_name, start_time)

    cursor_obj = conn.cursor()
    cursor_obj.execute(distinct_topics_query)
    topics = cursor_obj.fetchall()
    
    followers_list = []
    subscriber_list = []
    for topic in topics[0]:
        query_topic = '\'' + topic + '\''
        new_followers_query =  '''
                    with starting_followers as (
                        select follower_count
                        from chats_table_sentiment
                        where channel_name = {} 
                        and stream_date = {} 
                        and stream_topic = {} 
                        order by date asc
                        limit 1)

                    ,ending_followers as (
                        select follower_count
                        from chats_table_sentiment
                        where channel_name = {}
                        and stream_date = {}
                        and stream_topic = {} 
                        order by date desc
                        limit 1
                        )

                    select * from starting_followers
                    UNION ALL select * from ending_followers
                    '''.format(channel_name, start_time, query_topic, channel_name, start_time, query_topic)
        cursor_obj = conn.cursor()
        cursor_obj.execute(new_followers_query)
        follower_count = cursor_obj.fetchall()
        follower_change = follower_count[1][0] - follower_count[0][0]
        followers_list.append(follower_change)

        new_subscriber_query =  '''
                    with starting_subscribers as (
                        select subscriber_count
                        from chats_table_sentiment
                        where channel_name = {} 
                        and stream_date = {} 
                        and stream_topic = {} 
                        order by date asc
                        limit 1)

                    ,ending_subscribers as (
                        select subscriber_count
                        from chats_table_sentiment
                        where channel_name = {}
                        and stream_date = {}
                        and stream_topic = {} 
                        order by date desc
                        limit 1
                        )

                    select * from starting_subscribers
                    UNION ALL select * from ending_subscribers

                    '''.format(channel_name, start_time, query_topic, channel_name, start_time, query_topic)
        cursor_obj = conn.cursor()
        cursor_obj.execute(new_subscriber_query)
        subscriber_count = cursor_obj.fetchall()
        subscriber_change = subscriber_count[1][0] - subscriber_count[0][0]
        subscriber_list.append(followers_list)
        
    grouped_stream_query = '''with grouped_stream as (
                select channel_name, 
                stream_topic, 
                avg(viewer_count) as avg_viewers, 
                max(stream_length) as max_minutes,
                min(stream_length) as min_minutes,
                round(avg(message_sentiment),2) as average_sentiment
                from chats_table_sentiment
                where channel_name = {}
                and stream_date = {}
                group by channel_name, stream_topic
                )
                select channel_name,
                stream_topic,
                round(avg_viewers) as avg_viewers, 
                average_sentiment,
                (max_minutes - min_minutes) as time
                from grouped_stream
                
                '''.format(channel_name, start_time)
    
    df = pd.read_sql_query(grouped_stream_query, conn)
    df['subscriber_change'] = np.array(subscriber_list)
    df['followers_change'] = np.array(subscriber_list)
    
    
    if len(topics) > 1:
        for col in ['avg_viewers','average_sentiment','subscriber_change','followers_change']:
       
            df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
        df['score'] = df.apply(recommendation, axis = 1)
    else:
        df['score'] = 100
  
    return df



In [63]:
recommender_engine('tarik','2022-07-26 22:57:57.779134')

,channel_name,stream_topic,avg_viewers,average_sentiment,time,subscriber_change,followers_change,score
0,tarik,VALORANT,26813.0,None,1,1,1,100


In [ ]:
def avg_viewers(channel_name, stream_title):
	
    channel_name = '\'' + channel_name + '\''
    stream_title = '\'' + stream_title + '\''
    channel_name = '\'tarik\''


    conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')

    query = '''select avg(viewer_count) as avg_viewers 
                        from chats_table
                        where channel_name = {} and stream_title = {}
                        '''.format(channel_name)

    cursor_obj = conn.cursor()
    cursor_obj.execute(query)
    output = cursor_obj.fetchall()
    msg_count = output[0][0]
    conn.commit()
    conn.close()  


    return json.dumps([{
        'total_messages': msg_count }])

In [28]:
import sqlite3
import json

channel_name = '\'nickmercs\''


conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')

query = '''select
            *
            from chats_table_sentiment
            where channel_name = {} 
                    '''.format(channel_name)

cursor_obj = conn.cursor()
cursor_obj.execute(query)
output = cursor_obj.fetchall()
print(output)
conn.commit()
conn.close()  

#print(msg_count)





[('2022-07-26 22:58:09.861584', '2022-07-26 22:57:57.779134', 0, 'danny_dummy', 'I swear PK hasn’t even played Warzone since 2020', 'nickmercs', 'Call of Duty: Warzone', "Warzone Fraggin' | Code: MFAM | @NICKMERCS", 18271, 25129, 6522713, 80795, '5e211074-1d09-473e-92ea-6005a1ab3002', None), ('2022-07-26 22:58:09.861584', '2022-07-26 22:57:57.779134', 0, 'danny_dummy', '@dD0p3 LMAOOO', 'nickmercs', 'Call of Duty: Warzone', "Warzone Fraggin' | Code: MFAM | @NICKMERCS", 18271, 25129, 6522713, 80795, '5e211074-1d09-473e-92ea-6005a1ab3002', None), ('2022-07-26 22:58:09.861584', '2022-07-26 22:57:57.779134', 0, 'danny_dummy', 'Corina', 'nickmercs', 'Call of Duty: Warzone', "Warzone Fraggin' | Code: MFAM | @NICKMERCS", 18271, 25129, 6522713, 80795, '5e211074-1d09-473e-92ea-6005a1ab3002', None), ('2022-07-26 22:58:09.861584', '2022-07-26 22:57:57.779134', 0, 'danny_dummy', 'Follow Nick on Twitter for the latest news and updates about the Stream! https://twitter.com/NICKMERCS', 'nickmercs', 'C

In [27]:
import sqlite3
import json

channel_name = '\'tarik\''


conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')

query = '''with grouped_stream as (
                    select channel_name, 
                    stream_topic, 
                    avg(viewer_count) as avg_viewers, 
                    max(follower_count) as max_followers,
                    min(follower_count) as min_followers,
                    max(subscriber_count) as max_subscribers,
                    min(subscriber_count) as min_subscribers,
                    max(stream_length) as max_minutes,
                    min(stream_length) as min_minutes
                    from chats_table
                    where channel_name = {}
                    group by channel_name, stream_topic
                    )
            select channel_name,
            stream_topic,
            avg_viewers, 
            (max_followers - min_followers) as followers_gained,
            (max_subscribers - min_subscribers) as subscribers_gained,
            (max_minutes - min_minutes) as time
            from grouped_stream
                    '''.format(channel_name)

cursor_obj = conn.cursor()
cursor_obj.execute(query)
output = cursor_obj.fetchall()
#msg_count = output[0][0]
conn.commit()
conn.close()  

print(output)





[('tarik', 'VALORANT', 27946.530002871088, 471, 3801, 25)]


In [ ]:
INSERT INTO chats_table(
                                          date, 
                                          stream_date,
                                          stream_length,
                                          username, 
                                          message_text, 
                                          channel_name,
                                          stream_topic,
                                          stream_title,
                                          chatter_count,
                                          viewer_count,
                                          follower_count,
                                          subscriber_count,
                                          stream_id

In [1]:
import torch
import sqlite3
import json

def read_sentiments(model, tokenizer):
	conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')

	MAX_LEN = 160
	class_names = ['Negative', 'Neutral', 'Positive']
	cursor_obj = conn.cursor()
	cursor_obj.execute("select text, username from streamer_data_sqlite_chats order by RANDOM() limit 1")
	output = cursor_obj.fetchall()
	msg = output[0][0]
	uname = output[0][1]
	conn.commit()
	conn.close()  

	encoded_review = tokenizer.encode_plus(
		msg,
		max_length=MAX_LEN,
		add_special_tokens=True,
		return_token_type_ids=False,
		pad_to_max_length=True,
		return_attention_mask=True,
		return_tensors='pt',
		)

	input_ids = encoded_review['input_ids']
	attention_mask = encoded_review['attention_mask']

	output = model(input_ids, attention_mask)

	prob = torch.nn.functional.softmax(output, dim=1)
	top_prob = prob.topk(1, dim=1)[0].data[0].numpy()
	_, prediction = torch.max(output, dim=1)

	pred_class = class_names[prediction]
	pred_proba = format(top_prob[0], '.6f') if pred_class=='Positive' else format(top_prob[0] * -1, '.6f') if pred_class=='Negative' else 0
	# print(f'Review text: {msg}')
	# print(f'Sentiment  : {class_names[prediction]}')

	return json.dumps([{
		'sentiment_display': '<p>Sentiment for: <B>'+ msg + '</B> sent by user: <B>'+ uname + '</B> is: <B>' + pred_class + '</B></p>', 
		'prob_score' : pred_proba
	}])


Enter review text:test


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


-----------
tensor([1])
----->  tensor([[4.2926e-04, 9.9151e-01, 8.0604e-03]], grad_fn=<SoftmaxBackward0>)
Review text: test
Sentiment  : neutral


In [52]:
from load_sentiment_model import load_model
from transformers import BertTokenizer
import torch
import time
# import random
# import string
import sqlite3
import os

def message_sentiment(review_text, tokenizer, model, class_names, PRE_TRAINED_MODEL_NAME,MAX_LEN = 160):

    encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = encoded_review['input_ids']
    attention_mask = encoded_review['attention_mask']

    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    # print(f'_____: {_}')


    prob = torch.nn.functional.softmax(output, dim=1)
    top_prob = prob.topk(1, dim=1)[0].data[0].numpy()

    _, preds = torch.max(output, dim=1)

    pred_class = class_names[prediction]
    pred_proba = format(top_prob[0], '.6f') if pred_class=='positive' else format(top_prob[0] * -1, '.6f') if pred_class=='negative' else 0
    
    return pred_proba

In [126]:
#Pass to function
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = load_model()
PRE_TRAINED_MODEL_NAME = "veb/twitch-bert-base-cased-pytorch" #'models/veb/twitch-bert-base-cased-finetuned'
MAX_LEN = 160
class_names = ['negative', 'neutral', 'positive']
conn = sqlite3.connect('/home/w210/Twitch-chat-pioneers/src/front_end/db.sqlite3')
cursor_obj = conn.cursor()
cursor_obj.execute("""select username, channel_name, date, message_text from chats_sentiment_test
                    where sentiment_test_2 is NULL """)
output = cursor_obj.fetchall()
for review_text in output:
    sentiment = message_sentiment(review_text[3],
                                  tokenizer,
                                  model, 
                                  class_names, 
                                  PRE_TRAINED_MODEL_NAME,MAX_LEN = 160)
#     message_text = review_text[0].replace('\'',"\'\'")
#     message_text = message_text.replace('\"','\"\"')
#     message_text = message_text.replace('\r\n', '')
#     message_text = "\'" + message_text + "\'"
#     print(message_text)
    cursor_obj.execute("""UPDATE chats_sentiment_test
                        SET sentiment_test_2 = {}
                        WHERE username = {} and 
                        channel_name = {} and
                        date = {};""".format(sentiment,
                                                "\'" + review_text[0] + "\'",
                                                 "\'" + review_text[1] + "\'",
                                                 "\'" + review_text[2] + "\'" ))
    

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('no_uh_no_life', 'amouranth', '2022-07-25 23:46:10.415010', '!s')
no_uh_no_life
('no_uh_no_life', 'amouranth', '2022-07-25 23:46:10.415010', '@no_uh_no_life 🔞🥵HER SOCIAL MEDIA LINKS! 🍑💦 http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com')
no_uh_no_life
('no_uh_no_life', 'amouranth', '2022-07-25 23:46:10.424990', '!s')
no_uh_no_life
('no_uh_no_life', 'amouranth', '2022-07-25 23:46:10.424990', '@no_uh_no_life 🔞🥵HER SOCIAL MEDIA LINKS! 🍑💦 http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com http://downbad.com')
no_uh_no_life
('voxerguy13', 'amouranth', '2022-07-25 23:46:15.313397', '!watchtime\r\n')
voxerguy13
('voxerguy13', 'amouranth', '2022-07-25 23:46:15.317957', '!watchtime\r\n')
voxerguy13
('matteuzs00', 'amouranth', '2022-07-25 

('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', '!duo')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'NODDERS')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'Sully mad flicky')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', '!crosshair')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'stronger.')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'Kanye lebronJAM')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'S OMEGALULiguess L OMEGALULiguess')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'CSGO: cl_crosshairstyle 1;cl_fixedcrosshairgap -4.5;hud_showtargetid 0 | VALORANT: inner lines 1 1 4 2, everything else off')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'pepeCD')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'wait tarik do you listen to tame impala?')
akainu14x
('akainu14x', 'tarik', '2022-07-25 23:49:41.226180', 'poohDance poo

In [129]:
#cursor_obj.execute("""ALTER TABLE chats_sentiment_test ADD sentiment_test_2 REAL DEFAULT NULL""")
cursor_obj = conn.cursor()
cursor_obj.execute("""select sentiment_test_2 from chats_sentiment_test
                        where sentiment_test_2 is not NULL""")
output = cursor_obj.fetchall()
output

[(-0.943424,),
 (-0.943424,),
 (-0.957958,),
 (-0.957958,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.957719,),
 (0.957719,),
 (0.934341,),
 (0.934341,),
 (-0.885639,),
 (-0.885639,),
 (-0.874021,),
 (-0.874021,),
 (0.914569,),
 (0.892782,),
 (0.765386,),
 (0.763751,),
 (0.969073,),
 (0.969073,),
 (0.97031,),
 (0.97031,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.504608,),
 (0.423058,),
 (0.436836,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (-0.897221,),
 (-0.841304,),
 (-0.892922,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.992468,),
 (0.992468,),
 (0.996586,),
 (-0.914222,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.987199,),
 (0.987199,),
 (0.987199,),
 (0.987199,),
 (0.987199,),
 (-0.997618,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.0,),
 (0.996931,),
 (0.996931,),
 (0.996931,),
 (0.996931,),
 (0.996931,),
 (0.996931